In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../../../rings_varying_radius_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 64
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 64

MCMC_SIZE = 20
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-3

CUDA = torch.cuda.is_available()
PATH = 'ep-rad-init-z'
DEVICE = torch.device('cuda:1')

noise_sigma = torch.ones(1) * 0.05
if CUDA:
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (noise_sigma, N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)

optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (enc_eta, enc_eta, enc_z)

In [5]:
# enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
# enc_eta.load_state_dict(torch.load("../weights/enc-eta-%s" % PATH))

In [ ]:
from ag_ep_rad import *
train(models, EUBO_init_z, optimizer, Data, Model_Params, Train_Params)

epoch: 0\1000 (37s),  symKL_DB_eta: 234572.757,  symKL_DB_z: 240909.941,  gap: 475202.346,  loss: -75683.978,  ess: 2.412
epoch: 1\1000 (37s),  symKL_DB_eta: 38256.592,  symKL_DB_z: 144029.878,  gap: 182225.429,  loss: -32540.310,  ess: 3.012
epoch: 2\1000 (37s),  symKL_DB_eta: 19355.364,  symKL_DB_z: 74662.693,  gap: 93955.724,  loss: -25875.742,  ess: 3.532
epoch: 3\1000 (37s),  symKL_DB_eta: 15931.586,  symKL_DB_z: 42253.854,  gap: 58121.659,  loss: -21348.657,  ess: 3.903
epoch: 4\1000 (37s),  symKL_DB_eta: 16452.032,  symKL_DB_z: 28227.887,  gap: 44625.822,  loss: -22227.835,  ess: 4.135
epoch: 5\1000 (37s),  symKL_DB_eta: 17026.549,  symKL_DB_z: 20265.137,  gap: 37241.428,  loss: -22375.533,  ess: 4.282
epoch: 6\1000 (37s),  symKL_DB_eta: 17186.813,  symKL_DB_z: 15254.022,  gap: 32387.743,  loss: -22203.215,  ess: 4.388
epoch: 7\1000 (37s),  symKL_DB_eta: 16878.576,  symKL_DB_z: 12085.701,  gap: 28903.164,  loss: -22867.592,  ess: 4.456
epoch: 8\1000 (37s),  symKL_DB_eta: 16290.2

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)

In [ ]:
from ag_ep_rad import *

BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
Model_Params_Test = (noise_sigma, N, K, D, 10)
obs, metric_step, reused = test(models, EUBO_init_z, Data, Model_Params_Test, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')